First is the methodology test, then the code which turns it into a function

In [6]:
import csv
import pandas as pd
import numpy as np


In [ ]:
# Open the CSV file using pandas into a data frame
csv_file_path = '/Users/connorlockhart/Documents/GitHub/fall-2025-predicting-movie-success/Copy_popular_users_ratings_1k.csv'
df = pd.read_csv(csv_file_path)

#two ways to get the mean rating for each user

# Get the mean rating for each user
df['mean'] = (df['0.5']*0.5 + df['1']*1.0 + df['1.5']*1.5 + df['2']*2.0 + df['2.5']*2.5 + df['3']*3.0 + df['3.5']*3.5 + df['4']*4.0 + df['4.5']*4.5 + df['5']*5.0)/df['total']
#Round the mean to 2 decimal places
df['mean'] = df['mean'].round(2)


# Extract only the ratings columns for correlation calculation
ratings = df.iloc[:, 1:11]
print(ratings.head(n = 5))
#define the weights
weights = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
#take the dot product of ratings and weights
df['weighted_sum'] = ratings.dot(weights)
df['average'] = df['weighted_sum'] / df['total']
#Round the average to 2 decimal places
df['average'] = df['average'].round(2)

# Verify that both methods give the same result
print(df[['mean', 'average']].head(n=5))



   0.5   1  1.5    2  2.5    3  3.5    4  4.5    5
0   31  57   57  108  116  214  390  497  389  215
1   27  83   78   84  110  199  238  253  154   82
2   39  66  103  149  189  256  282  303   82   29
3   26  96  121  207  279  453  529  446  258  131
4   37  72  191  341  454  724  622  451  190   97
   mean  average
0  3.61     3.61
1  3.21     3.21
2  3.00     3.00
3  3.23     3.23
4  3.05     3.05


In [ ]:
#Define a function which automates the above process
def add_mean_column(input_csv, output_csv, start_col, end_col, col_name):
    # Read the input CSV file into a DataFrame
    df = pd.read_csv(input_csv)

    # Extract only the ratings columns for correlation calculation
    ratings = df.iloc[:, start_col:end_col]

    # Define the weights based on the rating values
    weights = [i * 0.5 for i in range(1, end_col - start_col + 1)]

    # Calculate the weighted sum and average
    df['weighted_sum'] = ratings.dot(weights)
    df[col_name] = df['weighted_sum'] / df['total']

    # Round the new column to 2 decimal places
    df[col_name] = df[col_name].round(2)

    # Optional
    # Drop the temporary weighted_sum column
    #df.drop(columns=['weighted_sum'], inplace=True)

    #Optional- reality check
    #print(df.head(n=5))

    # Save the updated DataFrame to the output CSV file
    df.to_csv(output_csv, index=False)

# Example usage
input_csv = '/Users/connorlockhart/Documents/GitHub/fall-2025-predicting-movie-success/Copy_popular_users_ratings_1k.csv'
output_csv = '/Users/connorlockhart/Documents/GitHub/fall-2025-predicting-movie-success/Copy_popular_users_ratings_1k_with_avg.csv'
add_mean_column(input_csv, output_csv, 1, 11, 'mean')



In [ ]:
#Standard Deviation Calculation
# Function to calculate standard deviation of ratings for each user
csv_file_path = '/Users/connorlockhart/Documents/GitHub/fall-2025-predicting-movie-success/Copy_popular_users_ratings_1k_with_avg.csv'
df = pd.read_csv(csv_file_path)

#function which subtracts the mean from each rating, squares the result, multiplies by the count, sums them, divides by total count, and takes the square root
#row is the input data frame, make sure the header is appropriately labeled
def calculate_std_dev(row):
    mean = row['mean']
    variance = (
        ((row['0.5'] * (0.5 - mean) ** 2) +
         (row['1'] * (1.0 - mean) ** 2) +
         (row['1.5'] * (1.5 - mean) ** 2) +
         (row['2'] * (2.0 - mean) ** 2) +
         (row['2.5'] * (2.5 - mean) ** 2) +
         (row['3'] * (3.0 - mean) ** 2) +
         (row['3.5'] * (3.5 - mean) ** 2) +
         (row['4'] * (4.0 - mean) ** 2) +
         (row['4.5'] * (4.5 - mean) ** 2) +
         (row['5'] * (5.0 - mean) ** 2))
        / row['total']
    )
    return np.sqrt(variance)    

df['std_dev'] = df.apply(calculate_std_dev, axis=1)
df['std_dev'] = df['std_dev'].round(2)
#print(df.head(n=20))